In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scvi
import numpy as np
import os
import squidpy as sq
import gseapy as gp
import anndata as ad
import gseapy as gp 
import cellcharter as cc 
import gseapy as gp 
import matplotlib.colors as mcolors
from gseapy.plot import barplot, dotplot


## Import files

In [ ]:
#### Please complete input PATH ####

samples = ['', '', '']

ad_dict = {}
for sample in samples:
    print(sample)
    adata = sc.read_h5ad('./'+ sample + '.h5ad')
    ad_dict[sample] = adata.copy()


## Cellcharter

In [ ]:
# refer from https://cellcharter.readthedocs.io/en/stable/

In [ ]:
adata = ad.concat(list(ad_dict.values()) , axis=0, merge='same', pairwise=True, index_unique='_')
adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
adata.uns['spatial_fov'] = {s: {} for s in adata.obs['sample'].unique()}

In [ ]:
scvi.settings.seed = 12345
scvi.model.SCVI.setup_anndata(
    adata, 
    batch_key='sample'
)

model = scvi.model.SCVI(adata)


In [ ]:
model.train(early_stopping=True, enable_progress_bar=True)


In [ ]:
adata.obsm['X_scVI'] = model.get_latent_representation(adata).astype(np.float32)

In [ ]:
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='generic', delaunay=True, spatial_key='spatial')
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='generic', delaunay=True, spatial_key='spatial')


In [ ]:
cc.gr.remove_long_links(adata)


In [ ]:
cc.gr.aggregate_neighbors(adata, n_layers=3, use_rep='X_scVI', out_key='X_cellcharter', sample_key='sample')


In [ ]:
# autok = cc.tl.ClusterAutoK(
#     n_clusters=(1,18), 
#     max_runs=10, 
#     model_params=dict(
#         random_state=12345,
#         trainer_params=dict(accelerator='gpu', devices=1)
#     ))
# autok.fit(adata, use_rep='X_cellcharter')
# cc.pl.autok_stability(autok) 
# adata.obs['spatial_cluster'] = autok.predict(adata, use_rep='X_cellcharter') 


In [ ]:
# After autoK, we choose the cluster = 12

gmm = cc.tl.Cluster(n_clusters=12, random_state=12345) 
gmm.fit(adata, use_rep='X_cellcharter')
adata.obs['spatial_cluster'] = gmm.predict(adata, use_rep='X_cellcharter')



In [ ]:
### Please complete output PATH ###
adata.write_h5ad('./output.h5ad')